# Analyzing RCT data with Precision Adjustment

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data. This time, we will focus on the treatment number 2.
  

In [32]:
## loading the data
Penn <- as.data.frame(read.table("../../data/penn_jae.dat", header=T ))
n <- dim(Penn)[1]
p_1 <- dim(Penn)[2]
Penn<- subset(Penn, tg==2 | tg==0)
attach(Penn)

The following objects are masked from Penn (pos = 7):

    abdt, agegt54, agelt35, black, dep, durable, female, hispanic,
    husd, inuidur1, inuidur2, lusd, muld, nondurable, othrace, q1, q2,
    q3, q4, q5, q6, recall, tg




In [33]:
T2<- (tg==2)
summary(T2)

   Mode   FALSE    TRUE 
logical    3354    2428 

In [34]:
head(Penn)

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,10824,0,18,18,0,0,0,0,2,0,...,1,0,0,0,0,0,0,0,1,0
2,10635,2,7,3,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,10824,0,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
5,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10,10754,2,20,20,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13,10831,0,27,27,0,0,0,0,1,0,...,1,0,0,0,1,1,0,1,0,0


In [35]:
#summarize variables 
#summary(Penn)

### Model 
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicators,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, if the RCT assumptions hold rigorously.


We also consider interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, if the RCT assumptions hold rigorously.

Under RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.

### Analysis

We consider 

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)
* interactive regression adjustment (IRA) using Lasso

and carry out robust inference using the *estimatr* R packages. 

# Carry out covariate balance check

This is done using "lm_robust" command which unlike "lm" in the base command automatically does the correct Eicher-Huber-White standard errors, instead othe classical non-robus formula based on the homoscdedasticity command.

In [36]:
#install.packages("lmtest")
#install.packages("sandwich")

library(lmtest)
library(sandwich)

In [37]:
m <- lm(T2~(female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)

coeftest(m, vcov = vcovHC(m, type="HC1"))


t test of coefficients:

                       Estimate Std. Error t value  Pr(>|t|)    
(Intercept)           0.5171342  0.1568823  3.2963 0.0009856 ***
female                0.0056240  0.1324597  0.0425 0.9661352    
black                -0.0307771  0.0847262 -0.3633 0.7164293    
othrace              -0.4278818  0.3005502 -1.4237 0.1545993    
factor(dep)1          0.1576567  0.1851798  0.8514 0.3945992    
factor(dep)2         -0.1927958  0.1508926 -1.2777 0.2014067    
q2                   -0.1092161  0.1583768 -0.6896 0.4904762    
q3                   -0.1075663  0.1579864 -0.6809 0.4959890    
q4                   -0.0621302  0.1581168 -0.3929 0.6943797    
q5                   -0.0498684  0.1575034 -0.3166 0.7515450    
q6                   -0.2176336  0.1558290 -1.3966 0.1625831    
agelt35              -0.0643464  0.1326376 -0.4851 0.6276032    
agegt54               0.0259251  0.2395371  0.1082 0.9138171    
durable              -0.1256397  0.1763853 -0.7123 0.4763064    

In [38]:
# get same dataframe

X <- as.data.frame( model.matrix(m) )

no_col <- attributes(m$coefficients)$names[is.na(m$coefficients)]

X1  <- X[, -which(names( X ) %in% no_col ) ]

save( X1, file = "../../data/m_reg.RData")

In [13]:
X

,(Intercept),female,black,othrace,factor(dep)1,factor(dep)2,q2,q3,q4,q5,...,agelt35:agegt54,agelt35:durable,agelt35:lusd,agelt35:husd,agegt54:durable,agegt54:lusd,agegt54:husd,durable:lusd,durable:husd,lusd:husd
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,1,0,1,0,0
6,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
7,1,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
8,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Generating similar qr decomposition

#### Replicate how lm drop variables [as discussed here](https://stackoverflow.com/questions/35410769/remove-perfectly-multicollinear-variables-from-data-frame)

We see that that even though this is a randomized experiment, balance conditions are failed.

# 1.1 Model Specification

In [14]:
# model specifications


# no adjustment (2-sample approach)
formula_cl <- log(inuidur1)~T4

# adding controls

formula_cra <- log(inuidur1)~T4+ (female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2

# Omitted dummies: q1, nondurable, muld


ols.cl_reg <- lm(formula_cl)
ols.cra_reg <- lm(formula_cra)


ols.cl = coeftest(ols.cl_reg, vcov = vcovHC(ols.cl_reg, type="HC1"))
ols.cra = coeftest(ols.cra_reg, vcov = vcovHC(ols.cra_reg, type="HC1"))

print(ols.cl)
print(ols.cra)




t test of coefficients:

             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.056830   0.020955 98.1557  < 2e-16 ***
T4TRUE      -0.085455   0.035856 -2.3833  0.01719 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


t test of coefficients:

                       Estimate Std. Error t value  Pr(>|t|)    
(Intercept)           2.6330806  0.3680995  7.1532 9.707e-13 ***
T4TRUE               -0.0796801  0.0355909 -2.2388 0.0252143 *  
female               -0.1146093  0.3163904 -0.3622 0.7171880    
black                -0.8990265  0.2402463 -3.7421 0.0001845 ***
othrace              -2.6269216  0.4541551 -5.7842 7.729e-09 ***
factor(dep)1         -0.7196978  0.5812182 -1.2383 0.2156788    
factor(dep)2         -0.0405584  0.3598395 -0.1127 0.9102631    
q2                   -0.1596700  0.3704317 -0.4310 0.6664596    
q3                   -0.5398906  0.3700794 -1.4589 0.1446691    
q4                   -0.4333540  0.3711943 -1.1675 0.2430809    
q5 

In [16]:
# get same dataframe for ols.cl

X <- as.data.frame( model.matrix(ols.cra_reg) )

no_col <- attributes(ols.cra_reg$coefficients)$names[is.na(ols.cra_reg$coefficients)]

X1  <- X[, -which(names( X ) %in% no_col ) ]


names(X1)[2] <- "T4"
save( X1, file = "../../data/ols_cra_reg.RData")


In [17]:
class(X1)

[1] "data.frame"

In [18]:
head(X1)

,(Intercept),T4,female,black,othrace,factor(dep)1,factor(dep)2,q2,q3,q4,...,q6:agegt54,q6:durable,agelt35:durable,agelt35:lusd,agelt35:husd,agegt54:durable,agegt54:lusd,agegt54:husd,durable:lusd,durable:husd
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
6,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
# interactive regression model variables 

X <- model.matrix(~(female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)[,-1] #without intercept



The interactive specificaiton corresponds to the approach introduced in Lin (2013).

## 1.3 Interactive regression model

In [20]:
#interactive regression model

demean<- function(x){ x - mean(x)}
X = apply(X, 2, demean)

ols.ira_reg = lm(log(inuidur1) ~ T4*X) 
ols.ira= coeftest(ols.ira_reg, vcov = vcovHC(ols.ira_reg, type="HC1"))
print(ols.ira)





t test of coefficients:

                               Estimate Std. Error t value  Pr(>|t|)    
(Intercept)                   2.0576131  0.0207724 99.0553 < 2.2e-16 ***
T4TRUE                       -0.0755005  0.0356049 -2.1205 0.0340132 *  
Xfemale                      -0.6662823  0.4090214 -1.6290 0.1033843    
Xblack                       -0.8634862  0.2976698 -2.9008 0.0037385 ** 
Xothrace                     -3.8176881  0.9389101 -4.0661 4.856e-05 ***
Xfactor(dep)1                 0.0359264  0.6492658  0.0553 0.9558747    
Xfactor(dep)2                 0.2117556  0.4523267  0.4681 0.6397000    
Xq2                          -0.2546436  0.4564528 -0.5579 0.5769552    
Xq3                          -0.6212326  0.4560767 -1.3621 0.1732217    
Xq4                          -0.4799269  0.4572363 -1.0496 0.2939421    
Xq5                          -0.3718675  0.4549983 -0.8173 0.4138001    
Xq6                          -0.6770474  0.4532559 -1.4937 0.1353075    
Xagelt35                 

In [21]:
head(X)

,female,black,othrace,factor(dep)1,factor(dep)2,q2,q3,q4,q5,q6,...,agelt35:agegt54,agelt35:durable,agelt35:lusd,agelt35:husd,agegt54:durable,agegt54:lusd,agegt54:husd,durable:lusd,durable:husd,lusd:husd
1,-0.4040008,-0.1219847,-0.007256325,-0.1121789,0.8362424,-0.2037654,-0.2355364,-0.2259267,0.7409296,-0.06295352,...,0,-0.07217101,-0.1321828,-0.1253187,-0.01804275,-0.03196705,-0.02647578,-0.05432438,-0.02804471,0
2,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,-0.06295352,...,0,-0.07217101,-0.1321828,-0.1253187,-0.01804275,-0.03196705,-0.02647578,-0.05432438,-0.02804471,0
3,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,0.7740733,-0.2590704,-0.06295352,...,0,-0.07217101,-0.1321828,-0.1253187,-0.01804275,-0.03196705,-0.02647578,-0.05432438,-0.02804471,0
4,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,0.7644636,-0.2259267,-0.2590704,-0.06295352,...,0,-0.07217101,-0.1321828,-0.1253187,-0.01804275,-0.03196705,-0.02647578,-0.05432438,-0.02804471,0
5,-0.4040008,-0.1219847,-0.007256325,0.8878211,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,-0.06295352,...,0,-0.07217101,-0.1321828,-0.1253187,0.98195725,0.96803295,-0.02647578,0.94567562,-0.02804471,0
6,0.5959992,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,-0.06295352,...,0,-0.07217101,-0.1321828,-0.1253187,-0.01804275,0.96803295,-0.02647578,-0.05432438,-0.02804471,0


In [23]:
# get same dataframe for ols.ira
S <- as.data.frame( model.matrix(ols.ira_reg) )

no_col <- attributes(ols.ira_reg$coefficients)$names[is.na(ols.ira_reg$coefficients)]

S1  <- S[, -which(names( S ) %in% no_col ) ]

names(S1)[2] <- "T4"
save( S1, file = "../../data/ols_ira_reg.RData")



# 1.3 Next we try out partialling out with lasso

In [24]:
T4 = demean(T4)
DX = model.matrix(~T4*X)[,-1]
head(DX)

,T4,Xfemale,Xblack,Xothrace,Xfactor(dep)1,Xfactor(dep)2,Xq2,Xq3,Xq4,Xq5,...,T4:Xagelt35:agegt54,T4:Xagelt35:durable,T4:Xagelt35:lusd,T4:Xagelt35:husd,T4:Xagegt54:durable,T4:Xagegt54:lusd,T4:Xagegt54:husd,T4:Xdurable:lusd,T4:Xdurable:husd,T4:Xlusd:husd
1,-0.342224,-0.4040008,-0.1219847,-0.007256325,-0.1121789,0.8362424,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,0.006174663,0.01093989,0.009060646,0.01859110,0.009597573,0
2,-0.342224,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,0.006174663,0.01093989,0.009060646,0.01859110,0.009597573,0
3,-0.342224,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,0.7740733,-0.2590704,...,0,0.02469865,0.04523612,0.04288706,0.006174663,0.01093989,0.009060646,0.01859110,0.009597573,0
4,0.657776,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,0.7644636,-0.2259267,-0.2590704,...,0,-0.04747236,-0.08694667,-0.08243163,-0.011868091,-0.02102716,-0.017415133,-0.03573327,-0.018447141,0
5,-0.342224,-0.4040008,-0.1219847,-0.007256325,0.8878211,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,-0.336049303,-0.33128407,0.009060646,-0.32363286,0.009597573,0
6,-0.342224,0.5959992,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,0.006174663,-0.33128407,0.009060646,0.01859110,0.009597573,0


In [25]:
library(hdm)

rlasso.ira = summary(rlassoEffects(DX, log(inuidur1), index = 1))

# rlassoEffects ( Partialling out )
# index = 1 (T4 treatment )
print(rlasso.ira)



ERROR: Error in library(hdm): there is no package called 'hdm'


In [27]:
# getting data
S = as.data.frame(DX)
save( S, file = "../../data/rlasso_ira_reg.RData")

In [28]:
head(S)

,T4,Xfemale,Xblack,Xothrace,Xfactor(dep)1,Xfactor(dep)2,Xq2,Xq3,Xq4,Xq5,...,T4:Xagelt35:agegt54,T4:Xagelt35:durable,T4:Xagelt35:lusd,T4:Xagelt35:husd,T4:Xagegt54:durable,T4:Xagegt54:lusd,T4:Xagegt54:husd,T4:Xdurable:lusd,T4:Xdurable:husd,T4:Xlusd:husd
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.342224,-0.4040008,-0.1219847,-0.007256325,-0.1121789,0.8362424,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,0.006174663,0.01093989,0.009060646,0.01859110,0.009597573,0
2,-0.342224,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,0.006174663,0.01093989,0.009060646,0.01859110,0.009597573,0
3,-0.342224,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,0.7740733,-0.2590704,...,0,0.02469865,0.04523612,0.04288706,0.006174663,0.01093989,0.009060646,0.01859110,0.009597573,0
4,0.657776,-0.4040008,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,0.7644636,-0.2259267,-0.2590704,...,0,-0.04747236,-0.08694667,-0.08243163,-0.011868091,-0.02102716,-0.017415133,-0.03573327,-0.018447141,0
5,-0.342224,-0.4040008,-0.1219847,-0.007256325,0.8878211,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,-0.336049303,-0.33128407,0.009060646,-0.32363286,0.009597573,0
6,-0.342224,0.5959992,-0.1219847,-0.007256325,-0.1121789,-0.1637576,-0.2037654,-0.2355364,-0.2259267,0.7409296,...,0,0.02469865,0.04523612,0.04288706,0.006174663,-0.33128407,0.009060646,0.01859110,0.009597573,0


### Results

In [30]:
install.packages("xtable")
library(xtable)
table<- matrix(0, 2, 4)
table[1,1]<-  ols.cl[2,1]
table[1,2]<-  ols.cra[2,1]
table[1,3]<-  ols.ira[2,1]
table[1,4]<-  rlasso.ira[[1]][1]

table[2,1]<-  ols.cl[2,2]
table[2,2]<-  ols.cra[2,2]
table[2,3]<-  ols.ira[2,2]
table[2,4]<-  rlasso.ira[[1]][2]


colnames(table)<- c("CL","CRA","IRA", "IRA w Lasso")
rownames(table)<- c("estimate", "standard error")
tab<- xtable(table, digits=5)
tab

print(tab, type="latex", digits=5)

Warning message:
"package 'xtable' is in use and will not be installed"


ERROR: Error in eval(expr, envir, enclos): objeto 'rlasso.ira' no encontrado


Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.


